In [1]:
from sympy import *
import numpy as np

In [2]:
ef, ece, vce, a = symbols('ef ece vce a')

C = 0.1*(exp(100*(ef - ece)) - 1) - a*(-4*ece**2 + 1)*(-atan(-0.5*vce)/atan(5) + 1)
pretty_print(C)

J = Matrix([C]).jacobian([ece, vce]).T
pretty_print(J)

H = hessian(C, [ece, vce])
pretty_print(H)

# J = Matrix([C.diff(ece), C.diff(vce)])
# pretty_print(J)

# H = Matrix([[C.diff(ece, ece), C.diff(ece, vce)], [C.diff(vce, ece), C.diff(vce, vce)]])
# pretty_print(H)

# Hinv = H.inv()
# pretty_print(Hinv)

    ⎛         2⎞ ⎛atan(0.5⋅vce)    ⎞        -100⋅ece + 100⋅ef      
- a⋅⎝1 - 4⋅ece ⎠⋅⎜───────────── + 1⎟ + 0.1⋅ℯ                  - 0.1
                 ⎝   atan(5)       ⎠                               
⎡        ⎛atan(0.5⋅vce)    ⎞         -100⋅ece + 100⋅ef⎤
⎢8⋅a⋅ece⋅⎜───────────── + 1⎟ - 10.0⋅ℯ                 ⎥
⎢        ⎝   atan(5)       ⎠                          ⎥
⎢                                                     ⎥
⎢                        ⎛         2⎞                 ⎥
⎢                 -0.5⋅a⋅⎝1 - 4⋅ece ⎠                 ⎥
⎢               ───────────────────────               ⎥
⎢               ⎛        2    ⎞                       ⎥
⎣               ⎝0.25⋅vce  + 1⎠⋅atan(5)               ⎦
⎡    ⎛atan(0.5⋅vce)    ⎞           -100⋅ece + 100⋅ef         4.0⋅a⋅ece        
⎢8⋅a⋅⎜───────────── + 1⎟ + 1000.0⋅ℯ                   ─────────────────────── 
⎢    ⎝   atan(5)       ⎠                              ⎛        2    ⎞         
⎢                                                     ⎝

: 

In [8]:
%matplotlib qt

import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import math

dataFol = '/home/haptix/haptix/Archive/biomechData/A_1/recordings/'
filler = '/experiments/1/'

jnts = ['indexAng', 'midAng', 'ringAng', 'pinkyAng', 'thumbInPlaneAng', 'thumbOutPlaneAng', 'wristFlex', 'wristRot']
jnts = [('Left', jnt) for jnt in jnts]

predsEtE = np.empty((0, len(jnts))); predsBio = np.empty((0, len(jnts))); targets = np.empty((0, len(jnts)))
for exp in os.listdir(dataFol):
    thesepredsEtE = pd.read_parquet(dataFol + exp + filler + 'pred_angles-EtE.parquet')
    thesepredsBio = pd.read_parquet(dataFol + exp + filler + 'pred_angles-biophysical.parquet')
    thesetargets = pd.read_parquet(dataFol + exp + filler + 'cropped_smooth_angles.parquet')

    thesepredsBio.loc[:, ('Left', 'thumbInPlaneAng')] = thesepredsBio.loc[:, ('Left', 'thumbInPlaneAng')] + math.pi
    thesepredsBio.loc[:, ('Left', 'wristRot')] = (thesepredsBio.loc[:, ('Left', 'wristRot')] + math.pi) / 2
    thesepredsBio.loc[:, ('Left', 'wristFlex')] = (thesepredsBio.loc[:, ('Left', 'wristFlex')] + math.pi / 2)

    thesepredsEtE.loc[:, ('Left', 'thumbInPlaneAng')] = thesepredsEtE.loc[:, ('Left', 'thumbInPlaneAng')] + math.pi
    thesepredsEtE.loc[:, ('Left', 'wristRot')] = (thesepredsEtE.loc[:, ('Left', 'wristRot')] + math.pi) / 2
    thesepredsEtE.loc[:, ('Left', 'wristFlex')] = (thesepredsEtE.loc[:, ('Left', 'wristFlex')] + math.pi / 2)

    thesetargets.loc[:, ('Left', 'thumbInPlaneAng')] = thesetargets.loc[:, ('Left', 'thumbInPlaneAng')] + math.pi
    thesetargets.loc[:, ('Left', 'wristRot')] = (thesetargets.loc[:, ('Left', 'wristRot')] + math.pi) / 2
    thesetargets.loc[:, ('Left', 'wristFlex')] = (thesetargets.loc[:, ('Left', 'wristFlex')] + math.pi / 2)

    predsEtE = np.vstack((predsEtE, thesepredsEtE[jnts].values))
    predsBio = np.vstack((predsBio, thesepredsBio[jnts].values))
    targets = np.vstack((targets, thesetargets[jnts].iloc[thesepredsBio.index].values))

predsEtE = np.clip((2*predsEtE - np.pi)/np.pi, -1, 1)
predsBio = np.clip((2*predsBio - np.pi)/np.pi, -1, 1)
targets = np.clip((2*targets - np.pi)/np.pi, -1, 1)

fig, axs = plt.subplots(len(jnts), 1)#, figsize=(15, 15))
for i in range(len(jnts)):
    axs[i].plot(predsEtE[:, i])
    axs[i].plot(predsBio[:, i])
    axs[i].plot(targets[:, i])
    axs[i].set_ylabel(jnts[i][1])

axs[0].legend(['EtE', 'Bio', 'Target'])
plt.show()

MSE_EtE = ((predsEtE - targets)**2).mean()
MSE_Bio = ((predsBio - targets)**2).mean()

MAE_EtE = np.abs(predsEtE - targets).mean()
MAE_Bio = np.abs(predsBio - targets).mean()

print(f'{MSE_EtE = :0.4f} | {MSE_Bio = :0.4f}\n{MAE_EtE = :0.4f} | {MAE_Bio = :0.4f}')

MSE_EtE = 0.0677 | MSE_Bio = 0.0722
MAE_EtE = 0.1584 | MAE_Bio = 0.1696
